In [1]:
# Imports
from numbers import Integral
from abc import ABC, abstractmethod

In [49]:
class BaseDescriptor(ABC):

    def __set_name__(self, owner_cls, name):
        self.name = name

    def __get__(self, obj, cls):
        if obj is None:
            return self
        return obj.__dict__.get(self.name, None)

    def __set__(self, obj, value) -> None:
        self.validator(value)
        obj.__dict__[self.name] = value

    @abstractmethod
    def validator(self, value):
        pass

In [77]:
class IntegerField(BaseDescriptor):

    def __init__(self, min_=None, max_=None):
        self._min = min_
        self._max = max_

    def validator(self, value):
        if not isinstance(value, Integral):
            raise ValueError(f'{self.name} should be an integer value')
        if self._min is not None and value < self._min:
            raise ValueError(f'{self.name} should be greater or equal than {self._min}')
        if self._max is not None and value > self._max:
            raise ValueError(f'{self.name} should be less or equal than {self._max}')

    

In [101]:
class CharField(BaseDescriptor):
    def __init__(self, min_=None, max_=None):
        self._min = min_ or 0
        self._max = max_

    def validator(self, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.name} should be an string value')
        if self._max is not None and len(value) > self._max:
            raise ValueError(f'{self.name} length should be less or equal than {self._min}')
        if self._min is not None and len(value) < self._min:
            raise ValueError(f'{self.name} length should be greater or equal than {self._min}')

In [61]:
# Manual testing
class Person:
    age = IntegerField(0, 10)


In [62]:
p = Person()

In [63]:
p.age = 5

In [64]:
p.age = 25

ValueError: age should be less or equal than 10

In [10]:
import unittest

In [11]:
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)

In [84]:
def create_type(class_name: str, *, attrs: dict = {}, mixins: tuple = () ) -> object:
    """ 
    Args:
        class_name: name of the class type that should be built
    Kwargs:
        attrs: dict of class attributes
        mixins: inheriting types
    Return:
        object
    """
    return type(class_name, mixins, attrs)

class TestIntegerField(unittest.TestCase):

    @staticmethod
    def create_test_class(min_, max_):
        return create_type('TestClass', attrs={'age': IntegerField(min_, max_)})

    def test_set_age_with_valid_params(self):
        min_ = 5
        max_ = 7
        o = self.create_test_class(min_, max_)()

        valid_values = range(min_, max_ +1)
        for i, value in enumerate(valid_values):
            with self.subTest(i):
                o.age = value
                self.assertEqual(value, o.age)

    def test_set_invalid_value_raises_value_error(self):
        min_ = -10
        max_ = 10
        o = self.create_test_class(min_, max_)()

        invalid_values = list(range(min_ -5, min_))
        invalid_values += list(range(max_ +1, max_ +5))
        invalid_values += ['a', None, 0j, 10.55, '2', (2,3)]

        for i, value in enumerate(invalid_values):
            with self.subTest(i):
                with self.assertRaises(ValueError):
                    o.age = value

    def test_class_get_return_integer_descriptor_instance(self):
        min_ = 5
        max_ = 7
        o = self.create_test_class(min_, max_)

        self.assertTrue(isinstance(o.age, IntegerField))

    def test_set_only_min_val(self):
        min_ = 0
        max_ = None
        o = self.create_test_class(min_, None)()
        values = range(min_, min_ + 100, 10)
        for i, value in enumerate(values):
            with self.subTest(i):
                o.age = value
                self.assertEqual(value, o.age)

    def test_set_only_max_val(self):
        min_ = None
        max_ = 100
        o = self.create_test_class(None, max_)()
        values = range(max_, max_ + 100, 10)
        for i, value in enumerate(values):
            with self.subTest(i):
                o.age = value
                self.assertEqual(value, o.age)

    def test_set_only_max_val(self):
        min_ = None
        max_ = None
        o = self.create_test_class(None, max_)()
        values = range(-100, 100, 10)
        for i, value in enumerate(values):
            with self.subTest(i):
                o.age = value
                self.assertEqual(value, o.age)
    

In [85]:
run_tests(TestIntegerField)

test_class_get_return_integer_descriptor_instance (__main__.TestIntegerField.test_class_get_return_integer_descriptor_instance) ... ok
test_set_age_with_valid_params (__main__.TestIntegerField.test_set_age_with_valid_params) ... ok
test_set_invalid_value_raises_value_error (__main__.TestIntegerField.test_set_invalid_value_raises_value_error) ... ok
test_set_only_max_val (__main__.TestIntegerField.test_set_only_max_val) ... ok
test_set_only_min_val (__main__.TestIntegerField.test_set_only_min_val) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.011s

OK


In [102]:
class TestCharFieldDescriptor(unittest.TestCase):
    
    @staticmethod
    def create_test_class(min_, max_):
        return create_type('TestClass', attrs={'val': CharField(min_, max_)})

    def test_set_age_with_valid_params(self):
        min_ = 1
        max_ = 7
        o = self.create_test_class(min_, max_)()

        valid_values = 'a', 'abc', 'abcd'
        for i, value in enumerate(valid_values):
            with self.subTest(i):
                o.val = value
                self.assertEqual(value, o.val)

    def test_set_invalid_value_raises_value_error(self):
        min_ = 0
        max_ = 1
        o = self.create_test_class(min_, max_)()

        invalid_values = ['ab', None, 0j, 10.55, '222', (2,3), 1, 2]

        for i, value in enumerate(invalid_values):
            with self.subTest(i):
                with self.assertRaises(ValueError):
                    o.val = value   

    def test_not_max_value_should_work(self):
        min_ = None
        max_ = None
        o = self.create_test_class(min_, max_)() 
        valid_values = 'a', 'abc', 'abcd'
        for i, value in enumerate(valid_values):
            with self.subTest(i):
                o.val = value
                self.assertEqual(value, o.val)

    def test_min_length_not_met_should_raise_value_error(self):
        min_ = 3
        max_ = 7
        o = self.create_test_class(min_, max_)()

        valid_values = 'ab', 'a'
        for i, value in enumerate(valid_values):
            with self.subTest(i):
                with self.assertRaises(ValueError):
                    o.val = value    

    def test_max_length_not_met_should_raise_value_error(self):
        min_ = 3
        max_ = 3
        o = self.create_test_class(min_, max_)()

        valid_values = 'ab', 'a'
        for i, value in enumerate(valid_values):
            with self.subTest(i):
                with self.assertRaises(ValueError):
                    o.val = value   

In [103]:
run_tests(TestCharFieldDescriptor)

test_min_length_not_met_should_raise_value_error (__main__.TestCharFieldDescriptor.test_min_length_not_met_should_raise_value_error) ... ok
test_not_max_value_should_work (__main__.TestCharFieldDescriptor.test_not_max_value_should_work) ... ok
test_set_age_with_valid_params (__main__.TestCharFieldDescriptor.test_set_age_with_valid_params) ... ok
test_set_invalid_value_raises_value_error (__main__.TestCharFieldDescriptor.test_set_invalid_value_raises_value_error) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.008s

OK
